# Subset Selection
## CMSE 381 - Fall 2023
## Oct 13,  2023. Lecture 17



In [15]:
# Everyone's favorite standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [16]:
# First, we're going to do all the data loading we've had for a while for this data set
auto = pd.read_csv('Auto.csv')
auto = auto.replace('?', np.nan)
auto = auto.dropna()
auto.horsepower = auto.horsepower.astype('int')

#this shuffles my data set in advance so that i don't need to worry about it later 
auto = auto.sample(frac=1).reset_index(drop=True)


auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,25.0,6,181.0,110,2945,16.4,82,1,buick century limited
1,13.0,8,302.0,130,3870,15.0,76,1,ford f108
2,14.0,8,400.0,175,4385,12.0,72,1,pontiac catalina
3,14.0,8,318.0,150,4096,13.0,71,1,plymouth fury iii
4,39.4,4,85.0,70,2070,18.6,78,3,datsun b210 gx


Let's try to run subset selection on the `auto` data set! We're going to use `cylinders`, `horsepower`, `weight`, and `acceleration` to predict `mpg`. 

In [17]:
inputvars = ['cylinders','horsepower','weight', 'acceleration']

The first tool we are going to use is the `itertools` package, which gives us a way to get subsets of whatever size we want using the `combinations` command.  

In [18]:
from itertools import combinations

The weird thing is it's an iterator, so if I just try to print out what I want, it's not helpful to me. 

In [19]:
combinations(inputvars,2)

But if I use it in a for loop it does what I want!

In [20]:
for x in combinations(inputvars,2):
    print(x)

('cylinders', 'horsepower')
('cylinders', 'weight')
('cylinders', 'acceleration')
('horsepower', 'weight')
('horsepower', 'acceleration')
('weight', 'acceleration')


Here's some code stolen from the last few days to run linear regression on a subset of the input variables. 

In [21]:
def myscore(df,listofvars, outputvar = 'mpg'):
    X = df[list(listofvars)]
    y = df[outputvar]
    
    #build linear regression model
    model = LinearRegression()
    

    #use 5-fold CV to evaluate model
    scores = cross_val_score(model, X,y, 
                             scoring='neg_mean_squared_error',
                             cv=5)

    #view mean absolute error
    return np.average(np.absolute(scores))
    

myvars = ('cylinders', 'acceleration')
myscore(auto,myvars)

24.87760952139212


&#9989; **<font color=red>Do this:</font>** Modify the code below as follows: 
- Set up two nested for loops to get every size $p = \{1,\cdots,4\}$ subset of my list of variables I want to use
- For each of these subsets, use the `myscore` function to get the k-fold CV error.
- Append it into a data frame as shown


In [22]:
myvars = []
myscores = []

for p in range(1, len(inputvars)+1):
    for subset in combinations(inputvars, p):
        score = myscore(auto, subset)
        myvars.append(subset)
        myscores.append(score)
        
myResults = pd.DataFrame({'Vars':myvars, 'Score':myscores})
myResults

,Vars,Score
0,"(cylinders,)",24.668419
1,"(horsepower,)",24.689481
2,"(weight,)",19.053274
3,"(acceleration,)",51.399379
4,"(cylinders, horsepower)",21.657183
5,"(cylinders, weight)",19.061222
6,"(cylinders, acceleration)",24.877610
7,"(horsepower, weight)",18.320551
8,"(horsepower, acceleration)",23.479927
9,"(weight, acceleration)",18.910563


We got all our main subsets, we're just missing a null model. This is the model that predicts the sample mean `mpg` for any input data. 

&#9989; **<font color=red>Do this:</font>** What is the MSE on our data set if we just predict the mean for every data point? Add this entry to your `myResults` data frame

*Hint: you can get a numpy array with every entry being the same output by using the `np.full` command.*

In [23]:
mean_prediction = auto['mpg'].mean()

null_predictions = np.full(len(auto), mean_prediction)

myscore = mean_squared_error(auto['mpg'], null_predictions)

In [24]:
newresult = pd.DataFrame({'Vars':['empty'], 'Score':[myscore]})

myResults = pd.concat([newresult,myResults],ignore_index = True)

# If you print it out now, you should have 16 models scored
myResults

,Vars,Score
0,empty,60.762738
1,"(cylinders,)",24.668419
2,"(horsepower,)",24.689481
3,"(weight,)",19.053274
4,"(acceleration,)",51.399379
5,"(cylinders, horsepower)",21.657183
6,"(cylinders, weight)",19.061222
7,"(cylinders, acceleration)",24.877610
8,"(horsepower, weight)",18.320551
9,"(horsepower, acceleration)",23.479927


&#9989; **<font color=red>Do this:</font>** What is the minimum score over all subsets? Which model makes it happen? The `idxmin` command will likely be useful for this. 

In [25]:
best_model_idx = myResults['Score'].idxmin()
best_model = myResults.loc[best_model_idx]

print(f"The minimum MSE score is {best_model['Score']:.4f}")
print(f"Achieved by using variables: {best_model['Vars']}")

The minimum MSE score is 18.3206
Achieved by using variables: ('horsepower', 'weight')


## Stretch project

Have some free time? 

&#9989; **<font color=red>Strech project:</font>** Figure out how to do this for forward selection. 

- First, try to write pseudo code for how to search through these subsets. 
- Then if you still have time, see if you can get it working on this set of four variables. 

In [26]:
# Your code here #



-----
### Congratulations, we're done!
Written by Dr. Liz Munch, Michigan State University

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.